# Purpose of Document:

1. See how different versions of ResNet compare and drop in performance with rotations of certain images

2. Davida's Number Images

In [3]:
!pip install torchvision

In [4]:
### LIBRARY IMPORTS
from google.colab import files
from torchvision import models
from PIL import Image
import os

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""
  ALGORITHM:

  1. We have the truth label
  2. I want to see the confidence of the model dropping when given a different orientation of that image
  3. See the differences when you freeze different layers of resnet 18 in the confidence dropping


"""

In [6]:
import string

# Generate labels from A to Z
letters = list(string.ascii_uppercase)

# Generate numbers from 0 to 9
numbers = list(range(10))

# Combine letters and numbers
all_labels = letters + numbers

# Create label mapping
label_mapping = {label: idx for idx, label in enumerate(all_labels)}

print(label_mapping)


{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 0: 26, 1: 27, 2: 28, 3: 29, 4: 30, 5: 31, 6: 32, 7: 33, 8: 34, 9: 35}


In [36]:
import torch
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Load pretrained ResNet-18
model = models.resnet18(pretrained=True)
model.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def get_image_confidence(model, image_path, true_label):
    img = Image.open(image_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = F.softmax(outputs, dim=1)

        # Convert true_label to its corresponding index using label_mapping
        if true_label.isdigit():
          true_label = int(true_label)
        elif true_label.islower():
          true_label = true_label.upper()

        target_index = label_mapping[true_label]

        confidence = probabilities[0][target_index].item()
    return confidence



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
folder_path = "/content/drive/MyDrive/iscr_image_dataset/letter_number_images_multi_fonts"

In [26]:
import re
pattern = r'^([A-Z])_(1|2|3|4|5)(_\d{1,3})?\.png$'

def sorting_key(img):
    match = re.match(pattern, img)
    if not match:
        return None
    alphabet = match.group(1)
    classname = int(match.group(2))
    orientation = int(match.group(3).replace("_", "")) if match.group(3) else 0
    return (alphabet, classname, orientation)

# Filter based on regex and then sort
all_files = os.listdir(folder_path)
sorted_files = sorted((file_name for file_name in all_files if sorting_key(file_name)), key=sorting_key)


print(len(sorted_files))

2470


In [19]:
count = 0
for file_name in os.listdir(folder_path):
  if count < 100:
    print(file_name)
    count += 1

A_9_100.png
V_4_260.png
O_7_340.png
O_6_100.png
6_1_120.png
A_8_40.png
T_6_260.png
T_7_160.png
small_p_9_80.png
small_g_4_100.png
T_7_340.png
T_10_160.png
T_7_40.png
V_4_300.png
small_c_1_340.png
V_4_240.png
small_a_3_300.png
T_7_300.png
K_3_60.png
Z_8_160.png
small_p_9_60.png
4_3_200.png
V_4_180.png
V_5_260.png
T_6_240.png
small_c_1_200.png
M_5_360.png
4_3_320.png
Z_8_60.png
X_10_20.png
T_7_200.png
small_z_2_260.png
X_10_100.png
4_3_140.png
small_a_3_180.png
small_c_1_40.png
A_8_300.png
small_p_9_140.png
R_1_240.png
T_7_360.png
small_z_2_80.png
X_10_140.png
4_3_360.png
M_5_20.png
6_1_80.png
small_z_2_120.png
small_a_2_320.png
A_9_180.png
6_1_20.png
O_7_260.png
small_z_3_280.png
A_9_360.png
O_6_240.png
small_a_3_160.png
V_5_180.png
small_g_4_140.png
O_7_100.png
4_3_40.png
6_1_340.png
A_8_200.png
O_6_260.png
small_g_4_340.png
small_p_9_200.png
O_6_340.png
R_1_100.png
V_4_320.png
T_10_180.png
O_6_140.png
V_4_20.png
T_6_200.png
A_8_220.png
small_a_3_240.png
A_9_220.png
small_g_4_40.png
K_

In [15]:
class_pattern = r'^(small_[a-z]+|[\dA-Z]+)_(1|2)(_\d{2,}|_[1-9])?\.png$'

def sorting_key(img):
    match = re.match(pattern, img)
    if not match:
      return None
    classname = int(match.group(2))
    orientation = int(match.group(3).replace("_", "")) if match.group(3) else 0
    return (classname, orientation)

sorted_images = sorted(filtered_images, key=sorting_key)

print(sorted_images)

TypeError: ignored

In [37]:
# RegEx Pattern

for file_name in os.listdir(folder_path):
  match = re.match(pattern, file_name)
  if match:
    label = match.group(2)
    if match.group(3):  # if orientation is present
        orientation = int(match.group(3).replace("_", ""))
    else:
        orientation = 0  # default orientation
    image_path = os.path.join(folder_path, file_name)
    confidence = get_image_confidence(model, image_path, label)  # Adjust function to use label appropriately

    if file_name not in results:
        results[file_name] = {}
    results[file_name][orientation] = confidence

KeyboardInterrupt: ignored

In [ ]:
# WE WILL DO THIS FOR 100 IMAGES ONLY:



In [15]:
print(results)

{}


In [ ]:
# Plotting
import matplotlib.pyplot as plt

plt.plot(orientations, avg_confidences)
plt.xlabel('Rotation Angle')
plt.ylabel('Model Confidence')
plt.title('Model Confidence vs. Image Rotation')
plt.show()
